In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from IPython.display import clear_output
from matplotlib.pyplot import imshow
import pandas as pd
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.metrics import *
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import backend as k
from tqdm.auto import tqdm
import tensorflow_addons as tfa

## Reading data

In [2]:
train_df = pd.read_csv('./data/train.csv')
train_df.head()

,id,path,label
0,TRAIN_000,./train/TRAIN_000.mp4,3
1,TRAIN_001,./train/TRAIN_001.mp4,0
2,TRAIN_002,./train/TRAIN_002.mp4,1
3,TRAIN_003,./train/TRAIN_003.mp4,4
4,TRAIN_004,./train/TRAIN_004.mp4,4


In [3]:
len(train_df)

610

In [4]:
content = []
y = []

for i in tqdm(range(len(train_df))):
    file = train_df.loc[i, 'id']
    fname = f'{file}.mp4'
    label = train_df.loc[i, 'label']
    content.append(fname)
    y.append(label)

  0%|          | 0/610 [00:00<?, ?it/s]

In [5]:
content = np.array(content)
y = np.array(y)

In [6]:
content.shape, y.shape

((610,), (610,))

In [7]:
def read_frames(root_folder, arr, each_nth=10):
    videos=[]
    for j  in range(len(arr)):
        clear_output()
        print(np.round(100*j/len(arr),3))
            
        vcap=cv2.VideoCapture(root_folder+arr[j])
        success=True
  
        frames=[]
        cnt=0
        while success:
            try:
              success,image=vcap.read()
              cnt+=1
              if cnt%each_nth==0:
                image=resize(image,(128,128))
                # image = image / 255.
                frames.append(image)
            except Exception as e:
                print(e)
        videos.append(frames)
    
    return videos

In [8]:
def select_frames(frames_arr , n=10):
    videos=[]
    for i in range(len(frames_arr)):
        frames=[]
        for t in np.linspace(0, len(frames_arr[i])-1, num=n):
            frames.append(frames_arr[i][int(t)])
        videos.append(frames)
        
    videos = np.array(videos)
    print(videos.shape)
    return videos

In [9]:
X_frames = read_frames('./data/train/', content)
X_frames = select_frames(X_frames, 30)

99.836


(610, 30, 128, 128, 3)


In [10]:
_, xte, _, yte = train_test_split(X_frames, y, shuffle=True, test_size=0.2, random_state=42)

In [11]:
ytr = to_categorical(y, 5)
yte = to_categorical(yte, 5)

In [12]:
X_frames.shape, xte.shape, ytr.shape, yte.shape

((610, 30, 128, 128, 3), (122, 30, 128, 128, 3), (610, 5), (122, 5))

## Building model